In [1]:
%matplotlib inline
import seaborn as sns; #sns.set(style="ticks", color_codes=True)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import scipy
style.use('fast')
from datetime import datetime
from io import StringIO
import re
import numpy as np
import os
from tqdm import tqdm
import json as js
import codecs
import subprocess

In [2]:
sns.set_style('ticks',{'legend.frameon':True} )
sns.set_palette('colorblind')
FIG_SIZE = (12, 3)
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['text.usetex'] = True

plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 22
plt.rcParams['axes.labelsize']= 22


# LOAD DATA for PACKAGES

In [3]:
unique_lag=pd.read_csv('../../jsep2018/csv/packages/dependencies_version_lag.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
unique_lag.head(2)

,dependency,last_available,used_version,last_today,constraint,date,date_last_today,date_used_version,date_last_available,version_lag
0,0,0.0.0,0.0.0,0.0.0,*,2016-03-30,2014-04-01,2014-04-01,2014-04-01,"0,0,0"
1,0,0.0.0,0.0.0,0.0.0,*,2015-11-11,2014-04-01,2014-04-01,2014-04-01,"0,0,0"


In [4]:
dependencies=pd.read_csv('../../jsep2018/csv/packages/dependencies_npm_march.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
print(len(dependencies))
dependencies.head(2)

52840196


,package,version,dependency,kind,constraint,date,release_type
0,0-,0.0.1,tap,Development,^6.3.2,2017-03-06,initial
1,0-0,1.0.2,gulp,runtime,*,2016-12-04,initial


In [5]:
for x in ['dependency','constraint','date']:
    dependencies[x]=dependencies[x].apply(str)
    unique_lag[x]=unique_lag[x].apply(str)

merge version lag with depenedencies

In [7]:
deps=(dependencies.
              set_index(['dependency','constraint','date']).
              merge(
                  unique_lag.
                  set_index(['dependency','constraint','date']),
                  left_index=True, 
                  right_index=True, 
                  how='left').
              reset_index().
              dropna()
              #drop_duplicates()
             )
print(len(deps))
deps.head(2)

51029864


,dependency,constraint,date,package,version,kind,release_type,last_available,used_version,last_today,date_last_today,date_used_version,date_last_available,version_lag
143,0,*,2015-11-11,shaybagants-test,1.0.0,runtime,initial,0.0.0,0.0.0,0.0.0,2014-04-01,2014-04-01,2014-04-01,"0,0,0"
144,0,*,2015-11-11,shaybagants-test,2.0.0,runtime,major,0.0.0,0.0.0,0.0.0,2014-04-01,2014-04-01,2014-04-01,"0,0,0"


In [8]:
deps.to_csv('../../jsep2018/csv/packages/all_deps_lag.csv', sep=';', index=False)

# LOAD DATA for REPOS

In [9]:
unique_lag=pd.read_csv('../../jsep2018/csv/repos/dependencies_version_lag_repos.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
unique_lag.head(2)

,dependency,last_available,used_version,last_today,constraint,date,date_last_today,date_used_version,date_last_available,version_lag
0,0,0.0.0,0.0.0,0.0.0,0.0.0,2016-09-11,2014-04-01,2014-04-01,2014-04-01,"0,0,0"
1,0,0.0.0,0.0.0,0.0.0,0.0.0,2016-03-16,2014-04-01,2014-04-01,2014-04-01,"0,0,0"


In [10]:
repos=pd.read_csv('../../jsep2018/csv/repos/repos_use_npm_march.csv', sep=';', dtype=object, index_col=None,  error_bad_lines=False)
print(len(repos))
repos.head(2)

11311371


,repo,dependency,constraint,kind,date
0,brianmhunt/knockout-modal.git,gulp-tag-version,^1.1.0,development,2016-12-18
1,brianmhunt/knockout-modal.git,gulp-load-plugins,^0.6.0,development,2016-12-18


In [11]:
for x in ['dependency','constraint','date']:
    repos[x]=repos[x].apply(str)
    unique_lag[x]=unique_lag[x].apply(str)

In [12]:
deps=(repos.
            set_index(['dependency','constraint','date']).
            merge(
              unique_lag.
              set_index(['dependency','constraint','date']),
              left_index=True, 
              right_index=True, 
              how='left').
            reset_index().
            dropna()
            #drop_duplicates()
            )
print(len(deps))
deps.head(2)

11049760


,dependency,constraint,date,repo,kind,last_available,used_version,last_today,date_last_today,date_used_version,date_last_available,version_lag
28,0,0.0.0,2016-03-16,changexbc/vue-app.git,runtime,0.0.0,0.0.0,0.0.0,2014-04-01,2014-04-01,2014-04-01,"0,0,0"
29,0,0.0.0,2016-09-11,JupiterLikeThePlanet/Recipes.git,development,0.0.0,0.0.0,0.0.0,2014-04-01,2014-04-01,2014-04-01,"0,0,0"


In [13]:
deps.to_csv('../../jsep2018/csv/repos/all_repos_lag.csv', sep=';', index=False)